# Lesson 1: First look at MEG and EEG

Neurocampus course "Signals of the whole brain"

Daria Kleeva

dkleeva@gmail.com

February 18, 2025



## Time series

A **time series** is a signal measured over time.

In EEG/MEG, each channel records voltage or magnetic field changes as a function of time. So the raw recording is a set of time series.

### Core concepts

1) Continuous-time signal

2) Discrete-time signal

3) Analog signal

4) Digital signal

In practice for EEG/MEG:
- Physics in the head and sensors is **continuous** and **analog**.
- The acquisition system converts it into **discrete-time** and **digital** data.

### Examples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
x = np.random.randn(10)  
n = np.arange(len(x))

plt.figure(figsize=(8, 3))
plt.stem(n, x)
plt.title('Random signal')
plt.xlabel('Sample index n')
plt.ylabel('Amplitude')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Basic signal descriptors
print('Indices n:', n)
print('Amplitude values x[n]:', np.round(x, 3))
print('Amplitude range [min, max]:', [x.min(), x.max()])
print('Min amplitude:', x.min())
print('Max amplitude:', x.max())
print('Mean:', x.mean())
print('Variance:', x.var())

In a computer we cannot create a truly continuous signal, but we can use a very dense time grid as a practical approximation.

In [ ]:
fs = 2000
T = 1.0         
f = 10           

t_cont = np.linspace(0, T, int(fs * T), endpoint=False)
x_cont = np.sin(2 * np.pi * f * t_cont)

plt.figure(figsize=(9, 3))
plt.plot(t_cont, x_cont, lw=1.5)
plt.title('Continuous-like oscillation: 10 Hz sine wave')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

<div style="color: blue;">
Try:

- To change frequency;
- To change amplitude;
- To add phase;
- To add noise;
- To mix oscillations.
</div>

In [ ]:
#... 

**Sampling**  (continuous-time -> discrete-time) is the process of measuring the signal at specific time points.

In [ ]:
fs = 40

t_cont = np.linspace(0, T, int(fs * T), endpoint=False)
x_cont = np.sin(2 * np.pi * f * t_cont)

plt.figure(figsize=(9, 3))
plt.plot(t_cont, x_cont, lw=1.5)
plt.title('Continuous-like oscillation: 10 Hz sine wave')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Aliasing effect:**  when we sample too slowly, a fast oscillation can look like a slower one.

In [ ]:
f_true = 18.0  
T = 1.0    

fs_ref = 4000
t_ref = np.linspace(0, T, int(fs_ref * T), endpoint=False)
x_ref = np.sin(2 * np.pi * f_true * t_ref)

# Sampling rates to compare
fs_list = [1000, 100, 60, 30, 20]  

#Computes the 'apparent' frequency we observe when sampling at fs Hz
def alias_frequency(f, fs):
    k = np.round(f / fs)
    f_alias = np.abs(f - k * fs)
    if f_alias > fs / 2:
        f_alias = fs - f_alias
    return f_alias

fig, axes = plt.subplots(len(fs_list), 1, figsize=(10, 8), sharex=True)

for ax, fs in zip(axes, fs_list):
    n = np.arange(0, int(T * fs))
    t_s = n / fs
    x_s = np.sin(2 * np.pi * f_true * t_s)

    ax.plot(t_ref, x_ref, color='0.85', lw=2, label='true signal (dense)')
    ax.stem(t_s, x_s, linefmt='C1-', markerfmt='C1o', basefmt='k-')

    f_a = alias_frequency(f_true, fs)
    nyq = fs / 2
    ax.set_title(f'fs = {fs} Hz, observed alias ~ {f_a:.1f} Hz')
    ax.grid(alpha=0.3)

axes[-1].set_xlabel('Time (s)')
for ax in axes:
    ax.set_ylabel('Amp')

plt.suptitle(f'Aliasing demo for true frequency f = {f_true} Hz', y=1.02)
plt.tight_layout()
plt.show()

**Nyquist-Shannon Sampling Theorem (practical form)**

If a continuous-time signal contains no frequencies above \(f_{\max}\), then it can be reconstructed from its samples without loss when:

$$
f_s > 2 f_{\max}
$$

- $2f_{\max}$ is the **Nyquist rate**.
- $f_s/2$ is the **Nyquist frequency**.

If $f_s < 2f_{\max}$, high-frequency components fold into lower frequencies (**aliasing**), and exact reconstruction is impossible.

**Rule of thumb:** to analyze frequencies up to $f_{\max}$, sample at least a little higher than $2f_{\max}$ (often 3-5x in practice), and use an anti-alias filter.

<div style="color: blue;">
How would you compute the period and the frequency from the given signal?
</div>

In [ ]:
#...

**Quantization** (analog -> digital) is the process of mapping the continuous amplitude values to a finite set of digital values. In practice, the continuous-time signal is sampled and quantized to create a discrete-time signal.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fs_dense = 2000
T = 1.0
f = 8
t = np.linspace(0, T, int(fs_dense * T), endpoint=False)
x = 0.9 * np.sin(2 * np.pi * f * t)

# Quantization function
def quantize(signal, n_levels, x_min=-1.0, x_max=1.0):
    step = (x_max - x_min) / (n_levels - 1)
    x_clip = np.clip(signal, x_min, x_max)
    x_q = np.round((x_clip - x_min) / step) * step + x_min
    return x_q, step

levels_list = [4, 8, 16]
fig, axes = plt.subplots(len(levels_list), 1, figsize=(10, 8), sharex=True)

for ax, L in zip(axes, levels_list):
    x_q, step = quantize(x, n_levels=L, x_min=-1, x_max=1)
    err = x - x_q
    mse = np.mean(err**2)

    ax.plot(t, x, color='0.75', lw=2, label='original')
    ax.plot(t, x_q, color='C3', lw=1.5, label=f'quantized ({L} levels)')
    ax.set_title(f'{L} levels | step = {step:.3f} | MSE = {mse:.5f}')
    ax.set_ylabel('Amplitude')
    ax.grid(alpha=0.3)
    ax.legend(frameon=False)

axes[-1].set_xlabel('Time (s)')
plt.suptitle('Quantization: fewer levels -> larger error', y=1.02)
plt.tight_layout()
plt.show()

Demo with the sound file:

In [ ]:
# %pip install librosa soundfile scipy

import librosa
from scipy.signal import resample_poly
from IPython.display import Audio, display

path = librosa.ex("robin") 
x, fs = librosa.load(path, sr=None, mono=True)

x = 0.95 * x / (np.max(np.abs(x)) + 1e-12)

print(f"Loaded: {path}")
print(f"fs={fs} Hz, duration={len(x)/fs:.2f} s")
print("Original:")
display(Audio(x, rate=fs))

plt.figure(figsize=(10, 4))
plt.plot(x, lw=1.5)
plt.title('Original sound')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

<div style="color: blue;">
Compute the representative frequency content of the sound.
</div>

In [ ]:
#...

Let's check ourselves!

**Spectrum (first intuition without formal definition)**

A time signal tells us how amplitude changes over time.  
A spectrum is another view of the same signal: it shows how strongly different oscillation rates (frequencies, in Hz) are present.

Important: the spectrum does **not** add new data; it is a re-expression of the same recording in frequency terms.  
So we use two complementary views:

- **Time domain:** when changes happen
- **Frequency domain (spectrum):** which rhythms are present and how strong they are

In [ ]:
plt.figure(figsize=(10, 4))
from scipy.signal import welch
f, Pxx = welch(x, fs=fs, nperseg=1024)
plt.plot(f, Pxx, lw=1.5)
plt.title('PSD')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.grid(alpha=0.3)
plt.tight_layout()

<div style="color: blue;">
Now let's listen to the sound under sampling and quantization.
</div>

In [ ]:
##...

## Signal building blocks

### Unit sample (impulse) signal

The **unit sample** (or discrete impulse) is denoted by $\delta[n]$:

$$
\delta[n] =
\begin{cases}
1, & n=0 \\
0, & n\neq 0
\end{cases}
$$

It is the simplest nonzero discrete-time signal: only one sample is equal to 1, all others are 0.

Why it is important:

- It is a basic building block for discrete signals.
- Any discrete signal can be represented as a weighted sum of shifted impulses.
- In system analysis, the response to $\delta[n]$ (impulse response) tells us how a linear system behaves.

Intuition: $\delta[n]$ is a “one-sample click” in time.

In [ ]:
n = np.arange(-10, 11)

delta = np.zeros_like(n, dtype=float)
delta[n == 0] = 1.0

plt.figure(figsize=(8, 3))
plt.stem(n, delta, linefmt='C0-', markerfmt='C0o', basefmt='k-')
plt.title('Unit sample (impulse) signal $\\delta[n]$')
plt.xlabel('Sample index n')
plt.ylabel('Amplitude')
plt.ylim(-0.1, 1.2)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

The shift:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


n = np.arange(-10, 11)

delta = np.zeros_like(n, dtype=float)
delta[n == 0] = 1.0

k_right = 3
k_left = 2

delta_right = np.zeros_like(n, dtype=float) 
delta_right[n == k_right] = 1.0

delta_left = np.zeros_like(n, dtype=float)   
delta_left[n == -k_left] = 1.0

fig, ax = plt.subplots(1, 3, figsize=(12, 3), sharey=True)

ax[0].stem(n, delta, linefmt='C0-', markerfmt='C0o', basefmt='k-')
ax[0].set_title(r'$\delta[n]$')

ax[1].stem(n, delta_right, linefmt='C1-', markerfmt='C1o', basefmt='k-')
ax[1].set_title(r'$\delta[n-3]$ (right shift)')

ax[2].stem(n, delta_left, linefmt='C2-', markerfmt='C2o', basefmt='k-')
ax[2].set_title(r'$\delta[n+2]$ (left shift)')

for a in ax:
    a.set_xlabel('Sample index n')
    a.set_ylim(-0.1, 1.2)
    a.grid(alpha=0.3)

ax[0].set_ylabel('Amplitude')
plt.tight_layout()
plt.show()

<div style="color: blue;">
Show that any discrete signal can be represented as a weighted sum of shifted impulses.
</div>

In [ ]:
#...

### Unit step signal

The **unit step** signal is denoted by $u[n]$:

$$
u[n] =
\begin{cases}
1, & n \ge 0 \\
0, & n < 0
\end{cases}
$$

It models a signal that is "off" before $(n=0$ and "on" from $n=0$ onward.

Why it is important:

- It is used to represent signal onset (sudden start).
- Many practical signals can be written using shifted/scaled step functions.
- It is closely related to the unit sample:

$$
\delta[n] = u[n] - u[n-1]
$$

(the impulse is the discrete difference of two steps).

In [ ]:
n = np.arange(-10, 11)

# Unit step: u[n] = 1 for n>=0, 0 otherwise
u = (n >= 0).astype(float)

plt.figure(figsize=(8, 3))
plt.stem(n, u, linefmt='C0-', markerfmt='C0o', basefmt='k-')
plt.title('Unit step signal $u[n]$')
plt.xlabel('Sample index n')
plt.ylabel('Amplitude')
plt.ylim(-0.1, 1.2)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

<div style="color: blue;">
Show that the impulse is the discrete difference of the two steps.
</div>

In [ ]:
##...

### Other basic signals (preview)

Besides impulse and step, other common signal representations are:

- **Constant (DC) signal:** $x[n] = C$  
  A fixed level over time (baseline component).

- **Ramp signal:** $r[n] = n\,u[n]$  
  Increases linearly with sample index (simple trend model).

- **Exponential signal:** $x[n] = a^n u[n]$  
  Models growth ($|a|>1$) or decay ($0<|a|<1$); with $a<0$, signs alternate.

## Typical structures in MNE Python

In [ ]:
import mne
from mne.datasets import sample

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = f'{sample_data_folder}/MEG/sample/sample_audvis_raw.fif'

In [ ]:
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)

In [ ]:
raw.info

In [ ]:
raw.first_samp

In [ ]:
raw.first_samp/raw.info['sfreq']

In [ ]:
raw.times

In [ ]:
fig=raw.plot_sensors()

In [ ]:
fig=raw.plot_sensors(ch_type='eeg', show_names=False)

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=True)

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=False, sphere=(0.03, 0.02, 0.01, 0.075))

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=False, sphere=0.17)

## Look at the data

In [ ]:
%matplotlib qt
raw.plot()

In [ ]:
raw_filt = raw.copy().filter(1., 40., fir_design='firwin')

In [ ]:
raw_filt.plot()

In [ ]:
raw.plot_psd(fmin=1., fmax=60., tmax=60., average=False)

In [ ]:
raw_filt.plot_psd(fmin=1., fmax=60., tmax=60., average=False)

## References and montage

raw_ref = raw.copy().set_eeg_reference(ref_channels=['EEG 001'])
raw_ref.plot()

Phase reversal demo:

In [ ]:
%matplotlib inline
# Time axis
fs = 500
T = 2.0
t = np.arange(int(fs * T)) / fs

# Base background activity
bg = 0.15 * np.sin(2 * np.pi * 8 * t)

# Sharp transient centered at t0 
t0 = 1.0
sigma = 0.015
sharp = np.exp(-0.5 * ((t - t0) / sigma) ** 2)

# Simulate 3 adjacent electrodes: F3 - C3 - P3
# Source maximal at C3, weaker at neighbors
F3 = bg + 0.4 * sharp
C3 = bg + 1.0 * sharp
P3 = bg + 0.4 * sharp

# Bipolar derivations
F3_C3 = F3 - C3
C3_P3 = C3 - P3

fig, ax = plt.subplots(2, 1, figsize=(11, 7), sharex=True)


ax[0].plot(t, F3, label='F3 (referential-like)')
ax[0].plot(t, C3, label='C3 (referential-like)')
ax[0].plot(t, P3, label='P3 (referential-like)')
ax[0].axvline(t0, color='k', ls='--', alpha=0.5)
ax[0].set_title('Sharp transient maximal at C3')
ax[0].set_ylabel('Amplitude')
ax[0].grid(alpha=0.3)
ax[0].legend(frameon=False, ncol=3)


ax[1].plot(t, F3_C3, label='F3 - C3', color='C1')
ax[1].plot(t, C3_P3, label='C3 - P3', color='C2')
ax[1].axvline(t0, color='k', ls='--', alpha=0.5)
ax[1].set_title('Bipolar channels: phase reversal around the sharp transient')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Amplitude')
ax[1].grid(alpha=0.3)
ax[1].legend(frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib qt

raw_bip = mne.set_bipolar_reference(raw.copy(), anode = ['EEG 001', 'EEG 004', 'EEG 008'], 
                                    cathode = ['EEG 004', 'EEG 008', 'EEG 018'])
raw_bip.plot()

In [ ]:
raw_av = raw.copy().set_eeg_reference(ref_channels='average')
raw_av.plot()

In [ ]:
import mne

# Example (assumes raw already loaded and EEG channels present)
montage = mne.channels.make_standard_montage("standard_1020")

raw.set_montage(montage, match_case=False, on_missing="warn")

print(raw.get_montage())
print(f"EEG channels: {len(mne.pick_types(raw.info, eeg=True))}")

# Optional quick check (spatial layout)
raw.plot_sensors(show_names=True)

In [ ]:
all_montages = mne.channels.get_builtin_montages()
print(f"Total built-in montages: {len(all_montages)}")
for name in all_montages:
    print(name)

In [ ]:
montage = mne.channels.make_standard_montage("standard_1020")

raw.set_montage(montage, match_case=False, on_missing="warn")

print(raw.get_montage())
print(f"EEG channels: {len(mne.pick_types(raw.info, eeg=True))}")


In [ ]:
mapping={'EEG 001': 'Fp1'}
raw.rename_channels(mapping)
raw.copy().pick('eeg').plot_sensors(show_names=True)

## Stimulus channel, events, and annotations

In [ ]:
raw.copy().pick(picks="stim").plot(start=10, duration=10)

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")

In [ ]:
events

In [ ]:
fig = mne.viz.plot_events(events, raw.info['sfreq'], raw.first_samp)

In [ ]:
testing_data_folder = mne.datasets.testing.data_path()
eeglab_raw_file = testing_data_folder / "EEGLAB" / "test_raw.set"
eeglab_raw = mne.io.read_raw_eeglab(eeglab_raw_file)
print(eeglab_raw.annotations)

In [ ]:
%matplotlib qt
eeglab_raw.plot()

In [ ]:
eeglab_raw.annotations.description

In [ ]:
eeglab_raw.annotations.onset

In [ ]:
eeglab_raw.annotations.duration

In [ ]:
events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw)
print(event_dict)
print(events_from_annot)

In [ ]:
%matplotlib inline
fig = mne.viz.plot_events(events_from_annot, eeglab_raw.info['sfreq'], eeglab_raw.first_samp)

In [ ]:
manual_annot = mne.Annotations(onset=[5, 41], duration=[16, 11], description=["Manual_marker"] * 2)
raw.set_annotations(manual_annot)
(manual_events, manual_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)
print(manual_event_dict)
print(manual_events)
fig = mne.viz.plot_events(manual_events, raw.info['sfreq'], raw.first_samp)


In [ ]:
%matplotlib qt
fig = raw.plot()
fig.fake_keypress("a")